In [158]:
import glob
import pandas as pd
import numpy as np
import environment2
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from collections import defaultdict

In [159]:
env = environment2.environment2()
user_list_2D = env.user_list_2D
user_list_3D = env.user_list_3D

In [160]:
user_list_2D[0]

'data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv'

In [161]:
fileNames2D=['data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv']

In [112]:
fileNames2D=user_list_2D

In [162]:
fileNames2D

['data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv']

In [163]:
#subset of users
fileNames3D=['data/NDSI-3D\\taskname_ndsi-3d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'data/NDSI-3D\\taskname_ndsi-3d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
 'data/NDSI-3D\\taskname_ndsi-3d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv',
 'data/NDSI-3D\\taskname_ndsi-3d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv',
 'data/NDSI-3D\\taskname_ndsi-3d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.csv']

In [164]:
def get_ROISubset(data):
    roi_subset=[]
    subset=1
    for i in range(len(data)):
        if data['State'][i]=='Sensemaking':
            if (i < (len(data) - 1)) and data['State'][i+1]!= 'Sensemaking':
                roi_subset.append(subset)
                subset=subset+1
            else:
                roi_subset.append(subset)
        else:
            roi_subset.append(subset)
    return roi_subset



In [165]:
def get_ROISubset_3D(data):
    roi_subset=[]
    subset=1
    for i in range(len(data)):
            if data['State'][i]=='Answering':
                    subset+=1
            if subset > 3:
                subset=3
            roi_subset.append(subset)
    return roi_subset

In [166]:
from collections import defaultdict
def get_probabilities(dataframe):
    dataframe['high-level-action']=np.zeros(len(dataframe))
    probabilities=[]
    map=dict()
    # print(dataframe.head(5))
    states=['Answering','Sensemaking','Foraging','Navigation']
    actions=['change','same']
    for state in states:
      for action in actions:
        map[state+action]=1


    current_state=None
    for i in range(len(dataframe)):
      if dataframe["State"][i]== current_state:
        dataframe["high-level-action"][i]="same"
      else:
        dataframe["high-level-action"][i]="change"
      current_state=dataframe["State"][i]

    for i in range(len(dataframe)):
      if dataframe['State'][i]=="Navigation":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Navigationchange']  + map['Navigationsame'])

      elif dataframe['State'][i]=="Sensemaking":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Sensemakingchange'] + map['Sensemakingsame'])

      elif dataframe['State'][i]=="Foraging":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Foragingchange'] + map['Foragingsame'])
      else:
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Answeringsame'] + map['Answeringchange'])
      probabilities.append(probs)
    dataframe['probabilities']=probabilities
    return dataframe, map

In [167]:
from collections import defaultdict
def get_probabilities_region(dataframe):
    df=dataframe
    probabilities=[]
    map=defaultdict(lambda: defaultdict(int))
    # print(dataframe.head(5))
    actions=['Answering','Sensemaking','Foraging','Navigation']
    states=['SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Northeast','West','Other','None']
    for state in states:
      for action in actions:
        map[state][action]=1

    for i in range(len(df)):
      sum = 0
      map[df['Most_frequent_region'][i]][df['State'][i]] +=1
      for act in map[df['Most_frequent_region'][i]]:
                sum += map[df['Most_frequent_region'][i]][act]
      probs=map[df['Most_frequent_region'][i]][df['State'][i]]   / sum
      probabilities.append(probs)
    df['probabilities']= probabilities
    return df, map

In [168]:
!pip install pymannkendall
from statsmodels.tsa.stattools import adfuller
import pymannkendall as mk

You should consider upgrading via the 'C:\Users\Nischal Aryal\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [169]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
        self.trend= None

    def ADF_Stationarity_Test(self, timeseries, printResults = True):
      #Dickey-Fuller test:
      adfTest = adfuller(timeseries, autolag='AIC')

      self.pValue = adfTest[1]

      if (self.pValue<self.SignificanceLevel):
          self.isStationary = True
      else:
          self.isStationary = False

      if printResults:
          dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
          #Add Critical Values
          for key,value in adfTest[4].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Augmented Dickey-Fuller Test Results:')
          print(dfResults)

    def Mann_Kendall_Stationarity_Test(self, timeseries, printResults = True):
      #Mann Kendall Trend Test:
      mktTest = mk.original_test(timeseries,self.SignificanceLevel)

      self.pValue = mktTest[2]
      self.trend= mktTest[0]
      self.isStationary = mktTest[1]

      if printResults:
          dfResults = pd.Series(mktTest[0:3], index=['Trend Type','Stationarity','P-Value'])
          #Add Critical Values
          for key,value in mktTest[3].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Mann Kendall Trend Test:')
          print(dfResults)

In [170]:
user=[]
ROI_Cycle=[]
SouthEastAnswering=[]
SouthEastSensemaking=[]
SouthEastForaging=[]
SouthEastNavigation=[]
SouthAnswering=[]
SouthSensemaking=[]
SouthForaging=[]
SouthNavigation=[]
OhioValleyAnswering=[]
OhioValleySensemaking=[]
OhioValleyForaging=[]
OhioValleyNavigation=[]
NorthernRockiesPlainsAnswering=[]
NorthernRockiesPlainsSensemaking=[]
NorthernRockiesPlainsForaging=[]
NorthernRockiesPlainsNavigation=[]
SouthWestAnswering=[]
SouthWestSensemaking=[]
SouthWestForaging=[]
SouthWestNavigation=[]
NorthWestAnswering=[]
NorthWestSensemaking=[]
NorthWestForaging=[]
NorthWestNavigation=[]
UpperMidwestAnswering=[]
UpperMidwestSensemaking=[]
UpperMidwestForaging=[]
UpperMidwestNavigation=[]
OtherAnswering=[]
OtherSensemaking=[]
OtherForaging=[]
OtherNavigation=[]
NoneAnswering=[]
NoneSensemaking=[]
NoneForaging=[]
NoneNavigation=[]

output_dataframe=pd.DataFrame(columns=['User','ROI_Cycle','SouthEastAnswering',
'SouthEastSensemaking',
'SouthEastForaging',
'SouthEastNavigation',
'SouthAnswering',
'SouthSensemaking',
'SouthForaging',
'SouthNavigation',
'OhioValleyAnswering',
'OhioValleySensemaking',
'OhioValleyForaging',
'OhioValleyNavigation',
'NorthernRockiesPlainsAnswering',
'NorthernRockiesPlainsSensemaking',
'NorthernRockiesPlainsForaging',
'NorthernRockiesPlainsNavigation',
'SouthWestAnswering',
'SouthWestSensemaking',
'SouthWestForaging',
'SouthWestNavigation',
'NorthWestAnswering',
'NorthWestSensemaking',
'NorthWestForaging',
'NorthWestNavigation',
'UpperMidwestAnswering',
'UpperMidwestSensemaking',
'UpperMidwestForaging',
'UpperMidwestNavigation',
'OtherAnswering',
'OtherSensemaking',
'OtherForaging',
'OtherNavigation',
'NoneAnswering',
'NoneSensemaking',
'NoneForaging',
'NoneNavigation'])


result_dataframe= pd.DataFrame(columns=['User','ROI_Cycle','SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Northeast','West','Other','None'])

for u in fileNames2D:
  print("###########################",u)
  index=[]

  all_dfs=[]
  maps=[]
  df = pd.read_csv(u)
  u=u.lstrip('data/NDSI-2D\\taskname_ndsi-2d-task_')
  #u=u.lstrip('data/NDSI-3D\\taskname_ndsi-3d-task_')
  u=u.rstrip('.csv')



  #df['ROI_Cycle']=get_ROISubset(df)
  df['ROI_Cycle']=get_ROISubset(df)

  # print(df.ROI_Cycle.unique())
  for k in df.ROI_Cycle.unique():
    dfs,mappings=get_probabilities_region((df.loc[df['ROI_Cycle'] == k]).reset_index(drop=True))
    mapper=dict()
    mapper['User']=u
    mapper['ROI_Cycle']=k
    print("Roi Cycle: #######################", k)
    for j in ['SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Northeast','West','Other','None']:
        #print("Region: ",j ,"Action Preferred: " ,max(mappings[j], key=mappings[j].get))
        maximum=max(mappings[j], key=mappings[j].get)
        if mappings[j][maximum] > 1 :
            action_preferred=maximum
        else:
            action_preferred='NA'
        mapper[j]=action_preferred
    some_df=pd.DataFrame([mapper])
    result_dataframe = result_dataframe.append(some_df, ignore_index=True)

        # series=(dfs.loc[dfs['Most_frequent_region'] == j])['probabilities']
        # if len(series) > 2 :
        #     sTest_state = StationarityTests()
        #     sTest_state.Mann_Kendall_Stationarity_Test(series.values, printResults = False)
        #     print("Region is: ", j, "  MKT Result is:  ", sTest_state.trend)
        # else :
        #      print("Region is: ", j, "  MKT Result is:  ","Not Enough Data")
    all_dfs.append(dfs)
    maps.append(mappings)
    ROI_Cycle.append(k)
    user.append(u)

########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv
Roi Cycle: ####################### 1
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv
Roi Cycle: ####################### 1
Roi Cycle: ####################### 2
Roi Cycle: ####################### 3
Roi Cycle: ####################### 4
Roi Cycle: ####################### 5
Roi Cycle: ####################### 6
Roi Cycle: ####################### 7
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv


C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_21824\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_21824\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_21824\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_21824\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed f

Roi Cycle: ####################### 1


C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_21824\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_21824\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_21824\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_21824\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed f

Roi Cycle: ####################### 2
Roi Cycle: ####################### 3
Roi Cycle: ####################### 4
Roi Cycle: ####################### 5
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv
Roi Cycle: ####################### 1
Roi Cycle: ####################### 2
Roi Cycle: ####################### 3
Roi Cycle: ####################### 4
Roi Cycle: ####################### 5
Roi Cycle: ####################### 6
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv
Roi Cycle: ####################### 1
Roi Cycle: ####################### 2
Roi Cycle: ####################### 3


C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_21824\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_21824\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_21824\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_dataframe = result_dataframe.append(some_df, ignore_index=True)
C:\Users\Nischal Aryal\AppData\Local\Temp\ipykernel_21824\1184691345.py:112: FutureWarning: The frame.append method is deprecated and will be removed f

In [171]:

result_dataframe.to_csv("data/NDSI-2D\\region_new-actions_roi_cycle.csv", index=False)
#output_dataframe.to_csv("data/NDSI-3D\\probability_distribution_roi_cycle_3d.csv", index=False)

In [172]:
result_dataframe.head(20)

,User,ROI_Cycle,SouthEast,South,OhioValley,NorthernRockiesPlains,SouthWest,NorthWest,UpperMidwest,Northeast,West,Other,None
0,userid_3abeecbe-327a-441e-be2a-0dd3763c1d45,1,NA,Foraging,NA,Sensemaking,NA,NA,Navigation,NA,NA,Sensemaking,Foraging
1,userid_44968286-f204-4ad6-a9b5-d95b38e97866,1,NA,Foraging,NA,NA,NA,Sensemaking,NA,NA,NA,Navigation,Foraging
2,userid_44968286-f204-4ad6-a9b5-d95b38e97866,2,NA,NA,NA,NA,NA,Navigation,NA,NA,NA,NA,Sensemaking
3,userid_44968286-f204-4ad6-a9b5-d95b38e97866,3,NA,NA,NA,Navigation,NA,Navigation,Foraging,NA,NA,Sensemaking,Foraging
4,userid_44968286-f204-4ad6-a9b5-d95b38e97866,4,NA,Foraging,NA,Navigation,NA,NA,NA,Navigation,NA,Navigation,NA
5,userid_44968286-f204-4ad6-a9b5-d95b38e97866,5,NA,NA,NA,Navigation,Navigation,NA,NA,Navigation,NA,Navigation,Navigation
6,userid_44968286-f204-4ad6-a9b5-d95b38e97866,6,NA,Foraging,NA,Foraging,NA,NA,NA,NA,NA,Sensemaking,Foraging
7,userid_44968286-f204-4ad6-a9b5-d95b38e97866,7,NA,Foraging,NA,Navigation,NA,NA,NA,NA,NA,Navigation,NA
8,userid_6d49fab8-273b-4a91-948b-ecd14556b049,1,NA,Foraging,NA,NA,NA,Sensemaking,NA,NA,Navigation,Navigation,Foraging
9,userid_6d49fab8-273b-4a91-948b-ecd14556b049,2,NA,Foraging,NA,Navigation,NA,Navigation,Navigation,NA,NA,Navigation,Navigation


In [173]:
user=[]
ROI_Cycle=[]
Foragingchange=[]
Foragingsame=[]
Foraging=[]
Sensemakingchange=[]
Sensemakingsame=[]
Sensemaking=[]
Navigationsame=[]
Navigationchange=[]
Navigation=[]


output_dataframe=pd.DataFrame(columns=['User','ROI_Cycle','Foragingchange','Foragingsame','Sensemakingchange','Sensemakingsame','Navigationsame','Navigationchange','Foraging','Navigation','Sensemaking'])
for u in fileNames2D:
  print("###########################",u)
  index=[]

  all_dfs=[]
  maps=[]
  df = pd.read_csv(u)
  u=u.lstrip('data/NDSI-2D\\taskname_ndsi-2d-task_')
  #u=u.lstrip('data/NDSI-3D\\taskname_ndsi-3d-task_')
  u=u.rstrip('.csv')



  #df['ROI_Cycle']=get_ROISubset(df)
  df['ROI_Cycle']=get_ROISubset(df)

  # print(df.ROI_Cycle.unique())
  for k in df.ROI_Cycle.unique():
    dfs,mappings=get_probabilities((df.loc[df['ROI_Cycle'] == k]).reset_index(drop=True))
    print("Roi Cycle: #######################", k, " TimeSpent: ############## ", dfs['IdleTime'].sum())
    for j in dfs.State.unique():
        series=(dfs.loc[(dfs['State'] == j) & (dfs["high-level-action"]=="same")])['probabilities']
        if len(series) > 2 :
            sTest_state = StationarityTests()
            sTest_state.Mann_Kendall_Stationarity_Test(series.values, printResults = False)
            print("State is: ", j, "  MKT Result is:  ", sTest_state.trend)
        else :
             print("State is: ", j, "  MKT Result is:  ","Not Enough Data")
    all_dfs.append(dfs)
    maps.append(mappings)
    ROI_Cycle.append(k)
    user.append(u)
  for k in range(len(maps)):
      #print("Cycle: ",k, "Foragingchange: ",maps[k]['Foragingchange']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
      Foragingchange.append(maps[k]['Foragingchange']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
      #print("Cycle: ",k, "Foragingsame: ",maps[k]['Foragingsame']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
      Foragingsame.append(maps[k]['Foragingsame']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']))
      Foraging.append((maps[k]['Foragingsame'] + maps[k]['Foragingchange']))


      #print("Cycle: ",k, "Navigationchange: ",maps[k]['Navigationchange']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
      Navigationchange.append(maps[k]['Navigationchange']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
      #print("Cycle: ",k, "Navigationsame: ",maps[k]['Navigationsame']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
      Navigationsame.append(maps[k]['Navigationsame']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']))
      Navigation.append((maps[k]['Navigationsame'] + maps[k]['Navigationchange']))


      #print("Cycle: ",k, "Sensemakingchange: ",maps[k]['Sensemakingchange']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']) )
      Sensemakingchange.append(maps[k]['Sensemakingchange']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
      #print("Cycle: ",k, "Sensemakingsame: ",maps[k]['Sensemakingsame']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']) )
      Sensemakingsame.append(maps[k]['Sensemakingsame']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
      Sensemaking.append((maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))


      print("##################################################")


output_dataframe['User']=user
output_dataframe['ROI_Cycle']=ROI_Cycle
output_dataframe['Foragingchange']=Foragingchange
output_dataframe['Foragingsame']=Foragingsame
output_dataframe['Foraging']=Foraging

output_dataframe['Sensemakingchange']=Sensemakingchange
output_dataframe['Sensemakingsame']=Sensemakingsame
output_dataframe['Sensemaking']=Sensemaking

output_dataframe['Navigationsame']=Navigationsame
output_dataframe['Navigationchange']=Navigationchange
output_dataframe['Navigation']=Navigation

########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv
Roi Cycle: ####################### 1  TimeSpent: ##############  175.86700000000002
State is:  Foraging   MKT Result is:   increasing
State is:  Navigation   MKT Result is:   increasing
State is:  Sensemaking   MKT Result is:   increasing
##################################################
########################### data/NDSI-2D\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv
Roi Cycle: ####################### 1  TimeSpent: ##############  36.94200000000001
State is:  Foraging   MKT Result is:   no trend
State is:  Navigation   MKT Result is:   no trend
State is:  Sensemaking   MKT Result is:   Not Enough Data
Roi Cycle: ####################### 2  TimeSpent: ##############  37.586999999999996
State is:  Navigation   MKT Result is:   no trend
State is:  Foraging   MKT Result is:   Not Enough Data
State is:  Sensemaking   MKT Result is:   no trend
Roi Cycle

In [174]:
output_dataframe.head(10)

,User,ROI_Cycle,Foragingchange,Foragingsame,Sensemakingchange,Sensemakingsame,Navigationsame,Navigationchange,Foraging,Navigation,Sensemaking
0,userid_3abeecbe-327a-441e-be2a-0dd3763c1d45,1,0.363636,0.636364,0.142857,0.857143,0.777778,0.222222,11,18,14
1,userid_44968286-f204-4ad6-a9b5-d95b38e97866,1,0.333333,0.666667,0.400000,0.600000,0.714286,0.285714,6,7,5
2,userid_44968286-f204-4ad6-a9b5-d95b38e97866,2,0.666667,0.333333,0.285714,0.714286,0.571429,0.428571,3,7,7
3,userid_44968286-f204-4ad6-a9b5-d95b38e97866,3,0.400000,0.600000,0.666667,0.333333,0.700000,0.300000,5,10,3
4,userid_44968286-f204-4ad6-a9b5-d95b38e97866,4,0.500000,0.500000,0.666667,0.333333,0.750000,0.250000,4,12,3
5,userid_44968286-f204-4ad6-a9b5-d95b38e97866,5,0.666667,0.333333,0.666667,0.333333,0.785714,0.214286,3,14,3
6,userid_44968286-f204-4ad6-a9b5-d95b38e97866,6,0.142857,0.857143,0.333333,0.666667,0.500000,0.500000,14,6,6
7,userid_44968286-f204-4ad6-a9b5-d95b38e97866,7,0.666667,0.333333,0.500000,0.500000,0.666667,0.333333,3,6,2
8,userid_6d49fab8-273b-4a91-948b-ecd14556b049,1,0.333333,0.666667,0.500000,0.500000,0.750000,0.250000,9,8,4
9,userid_6d49fab8-273b-4a91-948b-ecd14556b049,2,0.333333,0.666667,0.666667,0.333333,0.785714,0.214286,6,14,3


In [175]:
output_dataframe.to_csv("data/NDSI-2D\\probability_distribution_roi_cycle.csv", index=False)
#output_dataframe.to_csv("data/NDSI-3D\\probability_distribution_roi_cycle_3d.csv", index=False)

In [89]:
for u in output_dataframe['User'].unique()[1:]:
    print(u)
    user_df=(output_dataframe.loc[output_dataframe['User']==u]).sort_values(by=['ROI_Cycle'])
    print(user_df['Foragingsame'].values)
    sTest_state = StationarityTests()
    sTest_state.Mann_Kendall_Stationarity_Test(user_df['Foragingsame'].values, printResults = False)
    print(sTest_state.trend)
    print("#############################")

userid_44968286-f204-4ad6-a9b5-d95b38e97866
[0.66666667 0.33333333 0.6        0.5        0.33333333 0.85714286
 0.33333333]
no trend
#############################
userid_6d49fab8-273b-4a91-948b-ecd14556b049
[0.66666667 0.66666667 0.75       0.6        0.5       ]
no trend
#############################
userid_72a8d170-77ae-400e-b2a5-de9e1d33a714
[0.75       0.33333333 0.6        0.5        0.75       0.8       ]
no trend
#############################
userid_733a1ac5-0b01-485e-9b29-ac33932aa240
[0.33333333 0.5        0.77777778]
no trend
#############################


In [ ]:
# user=[]
# ROI_Cycle=[]
# Foragingchange=[]
# Foragingsame=[]
# Foraging=[]
# Sensemakingchange=[]
# Sensemakingsame=[]
# Sensemaking=[]
# Navigationsame=[]
# Navigationchange=[]
# Navigation=[]
#
#
# actions=['Answering','Sensemaking','Foraging','Navigation']
# states=['SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Other','None']
# for i in states:
#     for act in actions:
#           exec(f"{str(i)+str(act)} = {[]}")
# print(SouthEastAnswering)
# output_dataframe=pd.DataFrame(columns=['User','ROI_Cycle','Foragingchange','Foragingsame','Sensemakingchange','Sensemakingsame','Navigationsame','Navigationchange','Foraging','Navigation','Sensemaking'])
# for u in fileNames2D:
#   print("###########################",u)
#   index=[]
#
#   all_dfs=[]
#   maps=[]
#   all_dfs_region=[]
#   maps_region=[]
#   df = pd.read_csv(u)
#   u=u.lstrip('data/NDSI-2D\\taskname_ndsi-2d-task_')
#   #u=u.lstrip('data/NDSI-3D\\taskname_ndsi-3d-task_')
#   u=u.rstrip('.csv')
#
#
#
#   #df['ROI_Cycle']=get_ROISubset(df)
#   df['ROI_Cycle']=get_ROISubset(df)
#
#   # print(df.ROI_Cycle.unique())
#   for k in df.ROI_Cycle.unique():
#     dfs,mappings=get_probabilities((df.loc[df['ROI_Cycle'] == k]).reset_index(drop=True))
#     dfs_region,mappings_region=get_probabilities_region((df.loc[df['ROI_Cycle'] == k]).reset_index(drop=True))
#     all_dfs.append(dfs)
#     all_dfs_region.append(dfs_region)
#     maps.append(mappings)
#     maps_region.append(mappings_region)
#     ROI_Cycle.append(k)
#     user.append(u)
#   for k in range(len(maps)):
#       print("Cycle: ",k, "Foragingchange: ",maps[k]['Foragingchange']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
#       Foragingchange.append(maps[k]['Foragingchange']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
#       print("Cycle: ",k, "Foragingsame: ",maps[k]['Foragingsame']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']) )
#       Foragingsame.append(maps[k]['Foragingsame']/(maps[k]['Foragingsame'] + maps[k]['Foragingchange']))
#       Foraging.append((maps[k]['Foragingsame'] + maps[k]['Foragingchange']))
#
#
#       print("Cycle: ",k, "Navigationchange: ",maps[k]['Navigationchange']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
#       Navigationchange.append(maps[k]['Navigationchange']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
#       print("Cycle: ",k, "Navigationsame: ",maps[k]['Navigationsame']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']) )
#       Navigationsame.append(maps[k]['Navigationsame']/(maps[k]['Navigationsame'] + maps[k]['Navigationchange']))
#       Navigation.append((maps[k]['Navigationsame'] + maps[k]['Navigationchange']))
#
#
#       print("Cycle: ",k, "Sensemakingchange: ",maps[k]['Sensemakingchange']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']) )
#       Sensemakingchange.append(maps[k]['Sensemakingchange']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
#       print("Cycle: ",k, "Sensemakingsame: ",maps[k]['Sensemakingsame']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']) )
#       Sensemakingsame.append(maps[k]['Sensemakingsame']/(maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
#       Sensemaking.append((maps[k]['Sensemakingsame'] + maps[k]['Sensemakingchange']))
#
#
#       print("##################################################")
#
#
#   for k in range(len(maps_region)):
#       actions=['Answering','Sensemaking','Foraging','Navigation']
#       states=['SouthEast','South','OhioValley','NorthernRockiesPlains','SouthWest','NorthWest','UpperMidwest','Other','None']
#       for i in states:
#           sum=0
#           probabilities=[]
#           action=None
#           for act in mappings_region[i]:
#                     sum += mappings_region[i][act]
#                     probs=mappings_region[i][act]  / sum
#                     probabilities.append(probs)
#                     action=act
#
# output_dataframe['User']=user
# output_dataframe['ROI_Cycle']=ROI_Cycle
# output_dataframe['Foragingchange']=Foragingchange
# output_dataframe['Foragingsame']=Foragingsame
# output_dataframe['Foraging']=Foraging
#
# output_dataframe['Sensemakingchange']=Sensemakingchange
# output_dataframe['Sensemakingsame']=Sensemakingsame
# output_dataframe['Sensemaking']=Sensemaking
#
# output_dataframe['Navigationsame']=Navigationsame
# output_dataframe['Navigationchange']=Navigationchange
# output_dataframe['Navigation']=Navigation